In [ ]:
from numpy import zeros
from numpy import asarray

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Dense
from keras.layers import Flatten

from keras.models import Sequential

from keras.layers import Dense, LSTM, Dropout, Embedding,GRU
from keras.layers import Conv1D, MaxPooling1D

from keras.layers.embeddings import Embedding
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import regularizers,layers,constraints,initializers
from keras import initializers
from keras.engine.topology import Layer
from keras import backend as K
from keras.models import Model
from keras import utils



In [ ]:
#!pip install theano

In [ ]:
# Usual packages 
import os
import json
import requests
import datetime
import time
import joblib
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from pandas.io.json import json_normalize 
from itertools import chain

In [ ]:
parent_path =os.getcwd()
reviews_path = parent_path +"\\movie_rdata"
os.chdir(reviews_path)


# Process data from the collected Dir and make a df
print(reviews_path)

total_files_processed=0

for review_file in os.listdir(reviews_path):
    if review_file.endswith(".json"):
        with open(review_file) as infile:
            #print(review_file)
            jdata=json.load(infile)
            if(total_files_processed>0):
                movie_df=movie_df.append(json_normalize(jdata['reviews']))
                #print("coming here",total_files_processed)
            else:
                movie_df=json_normalize(jdata['reviews'])
                #print("first time",total_files_processed)
          
            total_files_processed=total_files_processed+1

            
print("Total files processed=",total_files_processed)
os.chdir(parent_path)

In [ ]:
# resetting index as each page index it as 0-9 from json collection method
movie_df=movie_df.reset_index(drop=True)

In [ ]:
movie_df

In [ ]:
movie_df['targetSentiment']=[0 if x>3 else 1 for x in movie_df['score']]

In [ ]:
pd.DataFrame(movie_df,columns=['targetSentiment','score']).head()

In [ ]:
## Making a copy and saving

In [ ]:
total_data=movie_df.copy(deep=True)

In [ ]:
columns_except_review_score=['createDate', 'displayImageUrl', 'displayName', 'hasProfanity',
       'hasSpoilers', 'isSuperReviewer', 'isVerified', 'rating','timeFromCreation', 'updateDate', 'user.accountLink',
       'user.displayName', 'user.realm', 'user.userId']

In [ ]:
total_data.drop(columns_except_review_score,inplace=True,axis=1)

In [ ]:
total_data.head()

In [ ]:
total_data.drop('score',inplace=True,axis=1)

In [ ]:
total_data=total_data.copy(deep=True)

In [ ]:
total_data.shape

In [ ]:
read_test_data = pd.read_csv("test-1566619745327.csv")

In [ ]:
read_test_data.drop('ReviewID',inplace=True,axis=1)

In [ ]:
read_test_data.insert(1,"targetSentiment",5555)

In [ ]:
total_data = pd.concat([total_data,read_test_data])

In [ ]:
total_data.shape

In [ ]:
total_data=total_data.reset_index(drop=True)

In [ ]:
total_data.head()

In [ ]:
total_data.tail()

In [ ]:
train_data=total_data[0:3000].copy(deep=True)

In [ ]:
insofe_data=total_data[3000:].copy(deep=True)

## Train and Test split

In [ ]:
X = train_data.drop(["targetSentiment"], axis = 1)

In [ ]:
y = train_data["targetSentiment"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
# Prepare tokenizer on ENTIRE DATASET including INSOFE DATA
t = Tokenizer()
t.fit_on_texts(total_data['review'])

print (t.word_index)

vocab_size = len(t.word_index) + 1
print (vocab_size)

In [ ]:

max_Review_len = 30
# no of words in sentence

# integer encode the documents
encoded_docs_Xtrain = t.texts_to_sequences(X_train['review'])
encoded_docs_Xtest = t.texts_to_sequences(X_test['review'])
encoded_docs_insofe = t.texts_to_sequences(insofe_data['review'])

padded_docs_Xtrain = pad_sequences(encoded_docs_Xtrain, maxlen=max_Review_len, padding='post')
padded_docs_Xtest = pad_sequences(encoded_docs_Xtest, maxlen=max_Review_len, padding='post')
padded_docs_insofe = pad_sequences(encoded_docs_insofe, maxlen=max_Review_len, padding='post')



In [ ]:
vocab_size = len(t.word_index) + 1

In [ ]:
print(vocab_size)

## Using Embedding layer

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    print("----"+word+"-----")
    print(coefs)
    print(embeddings_index[word])
    break
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))



In [ ]:

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Apply DL model

In [ ]:
# create the model lstm
embedding_vector_length = 50

model = Sequential()



#model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_Review_len,weights=[embedding_matrix],trainable=False))
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_Review_len))
model.add(Dropout(0.2))

model.add(GRU(units=50,dropout=0.2,recurrent_dropout=0.2))

#########################VER 2 file ###############
model.add(Dense(20, activation='tanh', kernel_initializer='normal'))
model.add(Dense(15, activation='tanh', kernel_initializer='normal'))

#########################VER 2 file ##############

if 1==0:
    model.add(Dense(80, activation='relu', kernel_initializer='normal'))
    model.add(Dropout(0.2))
    model.add(Dense(60, activation='tanh', kernel_initializer='normal'))
    model.add(Dropout(0.2))
    model.add(Dense(20, activation='relu', kernel_initializer='normal'))
    model.add(Dense(15, activation='relu', kernel_initializer='normal'))
    model.add(Dense(80, activation='relu', kernel_initializer='normal'))
    model.add(Dropout(0.2))
    model.add(Dense(60, activation='tanh', kernel_initializer='normal'))
    model.add(Dropout(0.2))
    model.add(Dense(20, activation='relu', kernel_initializer='normal'))
    model.add(Dense(15, activation='relu', kernel_initializer='normal'))
#model.add(GRU(units=max_Review_len,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))




In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
model.fit(padded_docs_Xtrain, y_train, validation_data=(padded_docs_Xtest, y_test), epochs=5, batch_size=64)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(padded_docs_Xtest, y_test, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
#results=model.predict_classes(padded_docs_insofe)
test_preds = model.predict_classes(padded_docs_Xtest,verbose=0)

In [ ]:
train_preds = model.predict_classes(padded_docs_Xtrain,verbose=0)

In [ ]:
from sklearn.metrics import f1_score
print("test f1 score",f1_score(y_test,test_preds,average=None))
print("train f1 score",f1_score(y_train, train_preds, average=None))

In [ ]:
## predict for Insofe class

In [ ]:
insofe_preds = model.predict_classes(padded_docs_insofe)

In [ ]:
pd.DataFrame(insofe_preds).to_csv("normal_play_LSTM_ver9.csv")

In [ ]:
  class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    :param kwargs:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = K.dot(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = K.dot(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return input_shape[0], input_shape[-1]
    
    def compute_output_shape(self, input_shape):
        """Shape transformation logic so Keras can infer output shape
        """
        return (input_shape[0], input_shape[-1])
    
    

In [ ]:


max_Review_len = 30
# no of words in sentence
embedding_vector_length = 50
embed_size=50
max_senten_len = max_Review_len
max_senten_num = 1
max_features = 6013
#vocab size

embedding_layer = Embedding(len(t.word_index) + 1,embed_size,weights=[embedding_matrix], \
                            input_length=max_senten_len, trainable=False)


#embedding_layer = Embedding(6013,embed_size, \
#                            input_length=max_senten_len,)


word_input = Input(shape=(max_senten_len,), dtype='float32')
word_sequences = embedding_layer(word_input)
word_lstm = Bidirectional(LSTM(150, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))(word_sequences)
word_dense = TimeDistributed(Dense(200, kernel_regularizer=regularizers.l2(0.01)))(word_lstm)
word_att = AttentionWithContext()(word_dense)
wordEncoder = Model(word_input, word_att)
# Sentence level attention model
sent_input = Input(shape=(max_senten_num, max_senten_len), dtype='float32')
sent_encoder = TimeDistributed(wordEncoder)(sent_input)
sent_lstm = Bidirectional(LSTM(150, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))(sent_encoder)
sent_dense = TimeDistributed(Dense(200, kernel_regularizer=regularizers.l2(0.01)))(sent_lstm)
sent_att = Dropout(0.5)(AttentionWithContext()(sent_dense))
preds = Dense(1, activation='softmax')(sent_att)
model = Model(sent_input, preds)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
import numpy as np
X_train_pd = np.reshape(padded_docs_Xtrain, (padded_docs_Xtrain.shape[0],1,padded_docs_Xtrain.shape[1]))
X_test_pd = np.reshape(padded_docs_Xtest, (padded_docs_Xtest.shape[0],1, padded_docs_Xtest.shape[1]))

In [ ]:
model.fit(X_train_pd, y_train, validation_data=(X_test_pd, y_test), epochs=2, batch_size=512)

In [ ]:
padded_docs_Xtrain.shape

In [ ]:
padded_docs_Xtrain.shape[0]

In [ ]:
padded_docs_Xtrain.shape[1]

In [ ]:
padded_docs_Xtrain.shape[1]

In [ ]:
padded_docs_insofe.shape

In [ ]:
insofe_pd = np.reshape(padded_docs_insofe, (padded_docs_insofe.shape[0],1,padded_docs_insofe.shape[1]))

In [ ]:
test_insofe_data_lstm_attn = model.predict(insofe_pd)

In [ ]:
 y_classes = np_utils.probas_to_classes(test_insofe_data_lstm_attn)

In [ ]:
## Save the model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_lstm_attn_withGlove.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_lstm_attn_withGlove.h5")
print("Saved model to disk")
 

In [ ]:
import copy
a = copy.deepcopy(test_insofe_data_lstm_attn)

In [ ]:
a

In [ ]:
labels = (a < 0.40).astype(np.int)

In [ ]:
pd.DataFrame(labels).to_csv("lstm_attn_ver_Glove_ver1.csv")